In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from IPython.core.pylabtools import figsize

In [2]:
rawtext = open("/Users/joe/Documents/study-group/data/alice_in_wonderland.txt", "r").read()[710:]
len(rawtext)

163107

In [3]:
split = int(0.9*len(rawtext))
trainraw = rawtext.lower()[:split]
testraw = rawtext.lower()[split:]

In [4]:
chars = sorted(list(set(rawtext.lower())))
charmap = {chars[i]:i for i in range(len(chars))}
len(chars)

59

In [5]:
seq_len = 25

def train_inpt_fn():
    def _gen():
        while True:
            seq_start = np.random.randint(0, len(trainraw)-seq_len-1)
            sequence = trainraw[seq_start:seq_start+seq_len]
            yield {"x":np.array([charmap[c] for c in sequence], dtype=np.int32)}, \
            charmap[trainraw[seq_start+seq_len]]
            
    ds = tf.data.Dataset.from_generator(_gen, ({"x":tf.int32}, tf.int32), 
                                        ({"x":[seq_len,]}, []))
    ds = ds.batch(100)
    ds = ds.prefetch(1)
    return ds.make_one_shot_iterator().get_next()

def test_inpt_fn():
    def _gen():
        for i in range(1000):
            seq_start = i*seq_len
            sequence = testraw[seq_start:seq_start+seq_len]
            yield {"x":np.array([charmap[c] for c in sequence], dtype=np.int32)}, \
                  charmap[testraw[seq_start+seq_len]]
            
    ds = tf.data.Dataset.from_generator(_gen, ({"x":tf.int32}, tf.int32), 
                                        ({"x":[seq_len,]}, []))
    ds = ds.batch(100)
    ds = ds.prefetch(1)
    return ds.make_one_shot_iterator().get_next()

In [6]:
train_inpt_fn()

({'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 25) dtype=int32>},
 <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>)

In [7]:
def model_fn(features, labels, mode, params):
    
    x_oh = tf.one_hot(features["x"], params["num_chars"])
    if labels is not None:
        y_oh = tf.one_hot(labels, params["num_chars"])

    if params["lmst"]:
        cell = tf.nn.rnn_cell.LSTMCell(params["hidden"])
    else:
        cell = tf.nn.rnn_cell.BasicRNNCell(params["hidden"])
    outputs, state = tf.nn.dynamic_rnn(cell, x_oh, dtype=tf.float32)
    final = outputs[:, -1, :]

    logits = tf.layers.dense(final, params["num_chars"])
    probs = tf.nn.softmax(logits)
    predictions = tf.argmax(probs, -1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, 
                            predictions={"probs":probs},
                            export_outputs={"probs":tf.estimator.export.PredictOutput(probs)})
    
    loss = tf.losses.softmax_cross_entropy(y_oh, logits)
    accuracy = tf.metrics.accuracy(labels, predictions)
    metrics = {"accuracy":accuracy}
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode,
                            loss=loss, eval_metric_ops=metrics)
        
    optimizer = tf.train.MomentumOptimizer(1e-3, 0.9)
    gs = tf.train.get_or_create_global_step()
    train_op = optimizer.minimize(loss, global_step=gs)
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, 
                        train_op=train_op, eval_metric_ops=metrics)

In [8]:
model = tf.estimator.Estimator(model_fn, "/Users/joe/Documents/study-group/logs/", 
                               params={"num_chars":len(chars), "lstm":False})

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/Users/joe/Documents/study-group/logs/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x18336f0208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
model.train(lambda: train_inpt_fn(), steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /Users/joe/Documents/study-group/logs/model.ckpt.
INFO:tensorflow:loss = 4.1186867, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /Users/joe/Documents/study-group/logs/model.ckpt.
INFO:tensorflow:Loss for final step: 3.99785.


In [10]:
def serving_input_fn():
    features = {"x":tf.placeholder(tf.int32, [1, seq_len])}
    return tf.estimator.export.build_raw_serving_input_receiver_fn(features, None)

In [11]:
serving_input_fn()()

ServingInputReceiver(features={'x': <tf.Tensor 'Placeholder_1:0' shape=(?, 25) dtype=int32>}, receiver_tensors={'x': <tf.Tensor 'Placeholder_1:0' shape=(?, 25) dtype=int32>}, receiver_tensors_alternatives=None)

In [12]:
model.export_savedmodel("/Users/joe/Documents/study-group/logs/", serving_input_fn())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['probs', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /Users/joe/Documents/study-group/logs/model.ckpt-10
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /Users/joe/Documents/study-group/logs/temp-b'1535942380'/saved_model.pb


b'/Users/joe/Documents/study-group/logs/1535942380'